# Execute this whole notebook and go back to `Session03-01-requests-json.ipynb`

In [1]:
import urllib.parse
import http.server
import socketserver
import json
import random
from pprint import pprint

class RandomDataGenerator:
    def __init__(self, seed, from_n, to_n):
        self.seed = seed
        self.from_n = from_n
        self.to_n = to_n
        self.rng = random.Random(self.seed)

    def get_next(self):
        return self.rng.randint(self.from_n, self.to_n)

    def get_range(self):
        return self.from_n, self.to_n

    def set_range(self, from_n, to_n):
        self.from_n = from_n
        self.to_n = to_n


rdg = RandomDataGenerator(1, 1, 10)

class CustomRequestHandler(http.server.BaseHTTPRequestHandler):
    def __init__(self, *args, **kwargs):
        self.routes = {
            'GET': {
                '/random_number': self.do_GET_random_number,
                '/random_range': self.do_GET_random_range,
            },
            'POST': {
                '/random_range': self.do_POST_random_range
            },
        }
        super(CustomRequestHandler, self).__init__(*args, **kwargs)


    def _send_response(self, data, status=200, content_type="application/json"):
        self.send_response(status)
        self.send_header("Content-Type", content_type)
        self.send_header("Content-Length", str(len(data)))
        self.end_headers()
        self.wfile.write(data)
        self.wfile.flush()


    def do_GET_random_number(self):
        data = {
            'number': rdg.get_next()
        }
        return self._send_response(json.dumps(data).encode('utf-8'))

    def do_GET_random_range(self):
        from_n, to_n = rdg.get_range()
        data = {
            'from_n': from_n,
            'to_n': to_n
        }
        return self._send_response(json.dumps(data).encode('utf-8'))

    def do_GET(self):
        url = urllib.parse.urlparse(self.path)
        handler = self.routes['GET'].get(url.path)
        if handler is not None:
            return handler()
        else:
            return self._send_response(f"404: {url}".encode('utf-8'), status=400)


    def do_POST_random_range(self):
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length)
        req_cont = json.loads(post_data.decode('utf-8'))
        rdg.set_range(req_cont['from_n'], req_cont['to_n'])
        return self.do_GET_random_range()


    def do_POST(self):
        url = urllib.parse.urlparse(self.path)
        handler = self.routes['POST'].get(url.path)
        if handler is not None:
            return handler()
        else:
            return self._send_response(f"404: {url}".encode('utf-8'), status=400)

PORT = 8891

if __name__ == "__main__":
    socketserver.TCPServer.allow_reuse_address = True
    with socketserver.TCPServer(("", PORT), CustomRequestHandler) as httpd:
        print("Hello. Serving at port", PORT)
        httpd.serve_forever()

Hello. Serving at port 8891


127.0.0.1 - - [06/May/2022 09:03:46] "GET /random_number HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2022 09:03:47] "GET /favicon.ico HTTP/1.1" 400 -
